## Experiment Notebook 




In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

import sys, os
from os.path import join, dirname
sys.path.insert(0, '/home/monte.flora/python_packages/scikit-explain')
sys.path.insert(0, '/home/monte.flora/python_packages/master/ml_workflow/')
path = dirname(dirname(os.getcwd()))
sys.path.insert(0, path)
from src.io.io import load_data_and_model

from skexplain import ExplainToolkit
from skexplain.common.importance_utils import to_skexplain_importance
from skexplain.common.multiprocessing_utils import run_parallel, to_iterator
from joblib import load
import pandas as pd

from ml_workflow.ml_methods import norm_aupdc, brier_skill_score, get_bootstrap_score
from ml_workflow import CalibratedPipelineHyperOptCV

import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import hyperopt.hp as hp
from tqdm import tqdm

In [ ]:
BASE_PATH = '/work/mflora/explainability_work/'
DATA_BASE_PATH = join(BASE_PATH, 'datasets')
MODEL_BASE_PATH = join(BASE_PATH, 'models')

## Create the Association between feature importance and model performance. 

In [ ]:
# Load in the respective ranking method results
def get_feature_importances(target, opt, method, results=None):
    
    time='first_hour'
    method = 'unnormalized'
    explainer = ExplainToolkit()
    if target == 'road_surface':
        ###opt = 'reduced'
        est_name = 'Random Forest'
        if opt == 'reduced':
            model_name = join(MODEL_BASE_PATH,'RandomForest_manualfeatures_12.joblib')
            model = load(model_name)['model'] 
            data = load(model_name)
            model = data['model'] 
            features = data['features']
            gini_values = model.base_estimator.named_steps['model'].feature_importances_
        else:
            model_name = join(MODEL_BASE_PATH,'JTTI_ProbSR_RandomForest.pkl')
            model = load(model_name)
            gini_values = model.feature_importances_
            features = PREDICTOR_COLUMNS
        
        pimp_backward_fname = join(DATA_BASE_PATH, f'perm_imp_{opt}_backward.nc')
        pimp_forward_fname = join(DATA_BASE_PATH, f'perm_imp_{opt}_forward.nc')
        ale_var_fname = join(DATA_BASE_PATH, f'ale_var_rf_{opt}.nc')
        shap_fname = join(DATA_BASE_PATH, f'shap_rf_{opt}.nc')

        gini_rank = to_skexplain_importance(gini_values,
                                       estimator_name='Random Forest', 
                                       feature_names=features, 
                                         method = 'gini')
        
    else:
        est_name = 'LogisticRegression'
        model_name = join(MODEL_BASE_PATH, 
                          f'LogisticRegression_first_hour_{target}_under_standard_{opt}.pkl')
        data = load(model_name)
        model = data['model']
        features = data['features']
    
        pimp_backward_fname = join(DATA_BASE_PATH,
                        f'permutation_importance_{target}_{time}_training_norm_aupdc{opt}backward.nc')
        pimp_forward_fname = join(DATA_BASE_PATH, 
                        f'permutation_importance_{target}_{time}_training_norm_aupdc{opt}forward.nc')
        ale_var_fname = join(DATA_BASE_PATH,
                        f'ale_var_results_all_models_{target}_{time}{opt}.nc')
        shap_fname = join(DATA_BASE_PATH,
                        f'shap_values_LogisticRegression_{target}_{time}{opt}.pkl')

        coefs = model.base_estimator.named_steps['model'].coef_[0]
        coef_rank = to_skexplain_importance(coefs,
                                       estimator_name='LogisticRegression', 
                                       feature_names=features, 
                                       method = 'coefs')
    
    sage_opt = 'reduced' if ('L1_based' in opt or opt=='reduced') else 'original'
    sage_fname = join(DATA_BASE_PATH,f'sage_results_{sage_opt}_{target}.nc')
    ale_var = explainer.load(ale_var_fname)
    pimp_backward = explainer.load(pimp_backward_fname)
    pimp_forward = explainer.load(pimp_forward_fname)
    
    with open(shap_fname, 'rb') as pkl_file:
        data = pickle.load(pkl_file)

    X = data['X']
    shap_vals = data['shap_values']
    shap_rank = to_skexplain_importance(shap_vals, 
                                     estimator_name=est_name, 
                                     feature_names=features, 
                                     method ='shap_sum')

    with open(sage_fname, 'rb') as f:
        sage_results = pickle.load(f)
    
    sage_rank = to_skexplain_importance(sage_results,
                                     estimator_name=est_name, 
                                     feature_names=features, 
                                     method = 'sage')
    
    data = [pimp_backward, 
            pimp_backward,
            pimp_forward, 
            pimp_forward,
            ale_var,  
            shap_rank,
            sage_rank,
           ]
    
    methods = ['multipass__backward', 
           'singlepass__backward', 
           'multipass__forward', 
           'singlepass__forward', 
           'ale_variance', 
           'shap_sum',
           'sage'
           ]

    if target == 'road_surface': 
        data.append(gini_rank)
        methods.append('gini')
    else:
        data.append(coef_rank)
        methods.append('coefs')
    
    # Turn the permutation importances into proper importance scores. 
    # Backward multipass and forward singlepass: 1 + (permute score - original score)
    original_score = pimp_backward[f'original_score__{est_name}'].values
    scores = 1.0 + (pimp_backward[f'multipass_scores__{est_name}'].values - original_score)
    pimp_backward[f'multipass_scores__{est_name}'] = (['n_vars_multipass', 'n_permute'], scores)
    
    original_score = pimp_forward[f'original_score__{est_name}'].values
    scores = 1.0 + (pimp_forward[f'singlepass_scores__{est_name}'].values - original_score)
    pimp_forward[f'singlepass_scores__{est_name}'] = (['n_vars_singlepass', 'n_permute'], scores)
  
    print(pimp_backward[f'multipass_scores__{est_name}'].shape)
    print(pimp_forward[f'multipass_scores__{est_name}'].shape)
    
    # Backward singlepass and forward multipass: original_score - permuted score
    original_score = pimp_backward[f'original_score__{est_name}'].values
    scores = original_score - pimp_backward[f'singlepass_scores__{est_name}'].values
    pimp_backward[f'singlepass_scores__{est_name}'] = (['n_vars_singlepass', 'n_permute'], scores)
    
    original_score = pimp_forward[f'original_score__{est_name}'].values
    scores = original_score - pimp_forward[f'multipass_scores__{est_name}'].values
    pimp_forward[f'multipass_scores__{est_name}'] = (['n_vars_multipass', 'n_permute'], scores)
    
    #data = [shap_rank, pimp_backward]
    #methods = ['shap_sum', 'multipass__backward']
    
    feature_importances = {m : pd.DataFrame(ds[f"{m.split('__')[0]}_scores__{est_name}"].values.T, 
                       columns = ds[f"{m.split('__')[0]}_rankings__{est_name}"]) 
                           for m,ds in zip(methods, data) }
    
    #for key in feature_importances.keys():
    #    print(key)
    #    print(f'{feature_importances[key]=}')
    
    if results is not None:
        feature_subsets = results['subsets']
        n_iter = len(feature_subsets)
        importances = {m : np.zeros((n_iter)) for m in methods}
         # Iterate over several thousand feature subsets
        for i, inds in enumerate(feature_subsets):
            for m in methods:
                # Sum up the importances from the different bootstrapping and then average. 
                # and then normalize by the feature set size. 
                if method == 'normalized':
                    factor =  float((len(features) - len(inds)) / len(features))
                elif method == 'normalized_divide':
                    factor = 1.0 / len(inds)
                else:
                    factor = 1.0
                
                val = np.sum(np.mean(feature_importances[m][inds], axis=0)) * factor
                
                importances[m][i] = val
    else:
        importances = None
    
    return feature_importances, importances 

In [ ]:
import traceback

targets= ['tornado', 'severe_hail', 'severe_wind', 'road_surface']
opts = ['L1_based_feature_selection_with_manual', '']
names = [ 'unnormalized']

def func(target, opt, name):
    if target == 'road_surface':
        opt = 'original' if opt == '' else 'reduced'
        fname = join(DATA_BASE_PATH,f'RF_road_surface_{opt}__scores.pkl')
    else:
        fname = join(DATA_BASE_PATH,f'LogisticRegression_first_hour_{target}_{opt}__scores.pkl')
    
    try:
        print('fname: ', fname)
        with open(fname, 'rb') as f:
            results = pickle.load(f)
        
        feature_importances, importances = get_feature_importances(target, opt, 
                                                                   results=results,
                                                                   method=name
                                                                   )
    except:
        print(traceback.format_exc())
        results=[]
        importances=[]
        
    try:
        scores = np.array(results['scores'])
    except:
        scores=[]
        
    data = {'scores' : scores, 
            'feature_importances': feature_importances, 
            'importances' : importances
    }
    
    save_fname = join(DATA_BASE_PATH,f'importances_{target}_{opt}_{name}.pkl')
    
    print(f'Writing {save_fname}...')
    with open(save_fname, 'wb') as f:
        pickle.dump(data, f) 

In [ ]:
run_parallel(func, 
             args_iterator=to_iterator(targets, opts, names), 
             n_jobs=len(targets)*len(opts)*len(names), 
             description='Feature Importance Experiment')

### Train models in the top and bottom 15 predictors. 

In [3]:
N_FEATURES = 15

In [13]:
def train_top_and_bottom_predictors(dataset):
    
    data = {}
    time = 'first_hour'
    opt = ''
    
    param_grid = {
            'tol' : hp.uniform('tol', 0.00001, 0.0001),
            'C': hp.lognormal('C', 0, 1.0),
            'solver' : 'saga',
            'max_iter' : hp.choice('max_iter', range(100,1000)),
                }
    
    save_fname = join(DATA_BASE_PATH, f'top_worst_predictors_{dataset}_{opt}_scores.pkl')
    model, X, y, dates, fti = load_data_and_model(dataset, 'original', DATA_BASE_PATH, MODEL_BASE_PATH, 
                                            return_dates=True)
    model_fname = join(MODEL_BASE_PATH,
                           f'LogisticRegression_first_hour_{dataset}_under_standard_{opt}.pkl')
    
    original_features = list(X.columns)
    #params = model[1].base_estimator.named_steps['model'].get_params()
    #params['max_iter'] = 300
    #params['n_jobs'] = 1
    
    params = {'solver' : 'saga', 'random_state' : 42}
    
    known_skew = np.mean(y)
        
    fname = join(DATA_BASE_PATH, f'importances_{dataset}_{opt}_unnormalized.pkl')
    with open(fname, 'rb') as f:
        data = pickle.load(f)     
    
    feature_importances = data['feature_importances']
    results = {m:{} for m in feature_importances.keys() }
    
    ### for m in feature_importances.keys():
    for m in ['sage']:
        
        print('*'*40)
        print(f'\nMethod : {m}\n')
        print('*'*40)
        imp = feature_importances[m]
        
        best_5 = list(feature_importances[m].columns[:N_FEATURES])
        worst_5 = list(feature_importances[m].columns[-N_FEATURES:])
        
        top_scores = []
        for i, _ in tqdm(enumerate(best_5), total=N_FEATURES, desc='Top Predictors'):
            estimator = LogisticRegression()
            
            clf = CalibratedPipelineHyperOptCV( base_estimator = estimator,
                                  param_grid = param_grid,
                                  resample='under',
                                  scaler='standard',
                                  imputer=None,
                                  hyperopt='atpe',
                                  n_jobs=1,
                                  max_iter=20,
                                  scorer_kwargs = {'known_skew': known_skew},
                                  cv = 'date_based',
                                  cv_kwargs = {'n_splits' : 5,
                                               'dates' : dates,
                                               'valid_size' : 20 },
                                  )
            
            
            feature_subset = best_5[:i+1]
            #print('\n', feature_subset)
            X_subset = X[feature_subset]
            clf.fit(X_subset, y, params=params)
            preds = clf.predict_proba(X_subset)[:,1]
            ds = get_bootstrap_score(y, preds, n_bootstrap=100, 
                                     known_skew = known_skew, 
                                     forecast_time_indices = fti.astype(float),
                                     metric_mapper={'naupdc' : norm_aupdc})
            top_scores.append(ds)
            
        bottom_scores = []
        for i, _ in tqdm(enumerate(worst_5[::-1]), total=N_FEATURES, desc='Bottom Predictors'):
            estimator = LogisticRegression()
            clf = CalibratedPipelineHyperOptCV( base_estimator = estimator,
                                  param_grid = param_grid,
                                  resample='under',
                                  scaler='standard',
                                  imputer=None,
                                  hyperopt='atpe',
                                  n_jobs=1,
                                  max_iter=20,
                                  scorer_kwargs = {'known_skew': known_skew },
                                  cv = 'date_based',
                                  cv_kwargs = {'n_splits' : 5,
                                               'dates' : dates,
                                               'valid_size' : 20 },
                                  )
            
            feature_subset = worst_5[::-1][:i+1]
            #print('\n', feature_subset)
            X_subset = X[feature_subset]
            clf.fit(X_subset, y, params=params)
            preds = clf.predict_proba(X_subset)[:,1]
            ds = get_bootstrap_score(y, preds, n_bootstrap=100, 
                                     known_skew = known_skew, 
                                     forecast_time_indices = fti.astype(float) ,
                                     metric_mapper={'naupdc' : norm_aupdc})
            
            bottom_scores.append(ds)
            
        #results[m]['top_scores'] = top_scores
        #results[m]['bottom_scores'] = bottom_scores
    
    with open(save_fname, 'rb') as f:
        existing_results = pickle.load(f)
    
    existing_results[m]['top_scores'] = top_scores
    existing_results[m]['bottom_scores'] = bottom_scores
    
    with open(save_fname, 'wb') as f:
        pickle.dump(existing_results, f)

In [14]:
#train_top_and_bottom_predictors('tornado')

****************************************

Method : sage

****************************************


Top Predictors:   0%|                                                                                                                                                 | 0/15 [00:00<?, ?it/s]

Unable to convert hyperparam results!


Top Predictors:   7%|█████████▏                                                                                                                               | 1/15 [00:12<02:53, 12.37s/it]

Unable to convert hyperparam results!


...

Bottom Predictors:  87%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                 | 13/15 [02:45<00:25, 12.96s/it]

Unable to convert hyperparam results!


Bottom Predictors:  93%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏        | 14/15 [02:58<00:13, 13.04s/it]

Unable to convert hyperparam results!


Bottom Predictors: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [03:11<00:00, 12.77s/it]


In [15]:
##train_top_and_bottom_predictors()
targets= ['tornado', 'severe_hail', 'severe_wind']

run_parallel(train_top_and_bottom_predictors, 
            args_iterator=to_iterator(targets), 
             n_jobs=len(targets), 
             description='Feature Importance Experiment')

Feature Importance Experiment:   0%|          | 0/3 [00:00<?, ?it/s]

****************************************

Method : sage

****************************************



Top Predictors:   0%|                                                                                                                                                 | 0/15 [00:00<?, ?it/s]

Unable to convert hyperparam results!
****************************************

Method : sage

****************************************



Top Predictors:   0%|                                                                                                                                                 | 0/15 [00:00<?, ?it/s]

...

Unable to convert hyperparam results!



Bottom Predictors:  93%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏        | 14/15 [03:26<00:15, 15.08s/it]

Unable to convert hyperparam results!



Bottom Predictors: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [03:41<00:00, 14.73s/it]


[None, None, None]